# Image filtering
Author: Julian Lißner

For questions and feedback write a mail to: [lissner@mib.uni-stuttgart.de](mailto:lissner@mib.uni-stuttgart.de)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append( 'provided_functions')
import result_check as check
import image_kernels as kernels

sys.path.append( 'incomplete_functions' )
import image_operations as operate
import binary_image_processing as binary
import image_conversion as convert

In [ ]:
def display( *images):
    """This function is defined for convenience and better readability of further blocks """
    n_images = len(images)
    if n_images > 3:
        n_row = 2
        n_col = np.ceil( n_images/2).astype(int)
    else:
        n_row = 1
        n_col = n_images
    fig, axes = plt.subplots( n_row, n_col, figsize=(n_col*6, n_row*6) )
    axes = axes.flatten()
    for i in range( n_images):
        axes[i].imshow( images[i], cmap='gray' )

    for ax in axes:
        ax.axis('off')
    fig.tight_layout()
    return fig, axes

- different kernel highlight different features
- low pass filter $\blacktriangleright$ retain low frequencies $\blacktriangleright$ smoothing of image (e.g. gaussian kernel)
- high pass filter $\blacktriangleright$ retain high frequencies $\blacktriangleright$ sharpen image (e.g. laplacian kernel)
- previously learned moving average (signal denoising) $\blacktriangleright$ low pass filter

## Edge detection
- an edge is defined as a large gradient between neighbouring pixels
- a kernel to detect these edges inspects neighbouring pixels
    - if neighbouring pixels are identical $\blacktriangleright$ no gradient $\blacktriangleright$ 0 activation of the kernel
    - if neighbouring pixels are different $\blacktriangleright$ gradient $\blacktriangleright$ activation of the kernel
- segmentation after filtering usually improves results

------
__Task:__ Create a kernel to find local vertical edges.

In [ ]:
image = 255-convert.load_grayscale( 'figures/house.jpg' )

edge_kernel = np.array( [[ #TODO,#TODO,#TODO]] ) #choose the values 0, 1 and -1
edges = operate.convolution( image, edge_kernel)

display( image, edges)

- note that depending on the orientation of the kernel the algorithm differs between 'positive' and 'negative' edges
- to highlight all edges, the absolute value of the image has to be taken
- __NOTE:__ for a nice highlight of the edges, the segmentation_threshold needs to be well tuned
- if there is too much 'noise' in the highlighted edges, the segmentation limit is too low
-----------
__Task:__ Apply binary segmentation to show _all_ major vertical edges, review the difference.

In [ ]:
all_edges =  #TODO #make 'negative' edges equally important

segmentation_threshold = #TODO
edge_highlight = #TODO( all_edges, #TODO) #segment the image with a good threshold
positive_edges = #TODO( edges, #TODO)

display( image, edges, edge_highlight, positive_edges )
vertical_edges = edge_highlight.copy()

- the steps to detect any directional edges are always equivalent 
- the major difference is the kernel and the segmentation threshold
----
__Task:__ Define and write yourself a function in 'image_operations.py' to execute the edge detection steps. <br> Do not forget to write a documentation for it!

In [ ]:
horizontal_edges = #TODO( image, edge_kernel, segmentation_threshold) #call your function
display( image, horizontal_edges)

---------
__Task:__ Highlight all major vertical edges.


In [ ]:
vertical_kernel = #TODO
vertical_edges = #TODO
display( image, horizontal_edges, vertical_edges)

## Sobel operators

- there are generally better ways to compute image gradients than a 1d vector
- the Sobel operator can be used to generally find good  gradients in $x$ and $y$ direction
- it is defined as
$$ H_H = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix} $$
$ H_V = H_H^{T} \qquad \quad \text{with} \cdot_V, \cdot_H, \text{ vertical/horizontal}  $
- if you inspect the kernel very closely, you will see that the sobol filter smoothes perpendicular to the edge detection direction
----- 
__Task:__ Apply both Sobel operators and compare it to the previous results. Note that the segmentation limit has to be chosen differently.

In [ ]:
vertical_filter =  #TODO
vertical_filter[ #TODO....
horizontal_filter = #TODO vertical_filter

sobel_ver = operate.#TODO #edge detection
sobel_hor = operate.#TODO #edge detection

fig, axes = display( vertical_edges, sobel_ver, horizontal_edges, sobel_hor)
for ax in axes[[0,2]]:
    ax.set_title( 'simple kernel')
for ax in axes[[1,3]]:
    ax.set_title( 'sobel operator')

## Laplace filter
- the Laplace filter can be used to highlight all contours
- it is able to find edges in both horizontal and vertical direction
- the laplace filter can be defined densely, considering values in a 45° angle as well
$$ F_d = \frac18 \begin{bmatrix} -1 & -1 & -1 \\ -1 & 8 & -1 \\ -1 & -1 & -1 \end{bmatrix} $$
- or it can be defined as a cross, considering only direct $x$ and $y$ neighbours
$$ F_c = \frac14 \begin{bmatrix} 0 & -1 & 0 \\ -1 & 4 & -1 \\ 0 & -1 & 0 \end{bmatrix} $$
- __Note__: If you see a black image in the Laplace-edges, turn down the segmentation threshold
----
__Task:__ Define the Laplacian kernel and apply it. Compare the results to the Sobel filter.

In [ ]:
laplacian = #TODO...
laplacian[ #TODO ...

unidirectional_edges =  #TODO

fig, axes = display( sobel_hor, sobel_ver,
            sobel_hor+ sobel_ver, unidirectional_edges)

fig.suptitle( 'detected edges in the image', y=0.95 )
titles = [ 'vertical', 'horizontal', 'horizontal+vertical', 'laplace' ]
for ax in axes:
    ax.set_title( titles.pop(0) )
    ax.axis( 'off' )

## RGB images
- RGB images are images with three channels, i.e. $A \in \mathbb R^{x \times y \times 3}$
- each color channel can have different highlights/contours
- for edge detection etc. it is generally advisable to work with the converted grayscale image

---
__Task:__ Apply edge detection on all three color channels _and_ on the grayscale image.<br>
Try out different filters for different results.

In [ ]:
image = convert.load_rgb( 'figures/mandrill.png' )
grayscale_image = convert.rgb_to_grayscale( image)

# 3 channels, R(ed), G(reen), B(lue)
# you might need to change the function, depending the name you used
threshold = 40
red_edges = operate.edge_detection( image[:,:,0], horizontal_filter, threshold )
green_edges = #TODO
blue_edges = #TODO

gray_edges = operate.edge_detection( grayscale_image, horizontal_filter, threshold )

fig, axes = display( image, grayscale_image, gray_edges, 
        red_edges, green_edges, blue_edges) 
axes[0].imshow( image)
titles = [ 'red channel', 'green channel', 'blue channel' ]
for ax in axes[-3:]:
    ax.set_title( titles.pop( 0) )

## Reducing unwanted noise
- noise reduction is a multi step process
- noise usually has high frequency $\blacktriangleright$ low pass to reduce it
- features are smeared out and can be enhanced again with a high pass filter
- denoising is made up of the following steps:
    - smoothing with e.g. a guassian kernel
    - edge detection/highlighting mask (computed on the smoothed image), e.g. with a laplacian kernel
    - enhancing of features by applying the mask

-----------
__Task:__ Reduce the noise in the given RGB image by applying the operations to each channel. Makre sure that the smoothing kernel is not too big.

In [ ]:
image = 255- convert.load_rgb( 'figures/lena.jpeg')
original_image = image.copy()
smoothed_image = np.zeros( image.shape) #holds the smoothed color channels
refined_image =  np.zeros( image.shape) #holds the fully processed image

smoothing_window = #TODO #adjust this parameter to get a nice result
gaussian = kernels.disc( smoothing_window, kind='gauss')
laplacian = kernels.laplace()
print(gaussian.sum())

for i in range( #TODO #loop through all channels
    smoothed_image[..., i] =  #TODO #smoothing/blurring
    refined_image[..., i]  = #TODO #highlight/contour detection

refined_image += smoothed_image #have the highlight be applied over the smoothed image


## plotting of results
fig, axes = plt.subplots( 1, 3, figsize=(15,5) )
axes[0].imshow( original_image)
axes[1].imshow( smoothed_image.astype(int))
axes[2].imshow( refined_image.astype(int))

titles = [ 'noisy image', 'smoothed image', 'refined image' ]
for ax in axes:
    ax.axis( 'off' )
    ax.set_title( titles.pop( 0 ) )


- even if the image does not look much different to us, for the machine it does
- when computing edges (below), you will see a major difference

## Computational tweak

- a kernel can be defined to smooth and detect edges in one convolution
- instead of a convolution of the image with both kernels, the kernels can be pre-convoluted
- thus we can save one costly computation of the image with the kernel in this example
- the kernels could be precomputed and stored in fourier space for more computational savings
----
__Task:__ Define yourself the smooth gradient kernel and apply it, compare it to a noisy gradient computation.

In [ ]:
image = convert.load_grayscale( 'figures/lena.jpeg')
laplacian = kernels.laplace()
gaussian = kernels.disc( #TODO, kind='gauss') 
kernel = #TODO #smooth gradient kernel

noisy_gradient  = operate.edge_detection( image, laplacian, #TODO)
smooth_gradient = #TODO #apply the previous edge detection function


display( original_image, refined_image.astype(int), noisy_gradient, smooth_gradient)


note how all the 'wrongly detected edges' (salt noise) are removed by smoothing, even though the images above look barely different to us humans.